#### 작업 노트북 
- feature 정리 
- 데이터 전처리
- 모델링
- 학습 및 추론
- 제품별 추천 결과 테이블

In [1]:
import pandas as pd
import warnings
import re 
from model.bow import BagOfWords
from model.tfidf import TfIdf
import yaml
warnings.filterwarnings("ignore")

C:\Users\gnsdl\AppData\Local\Temp\ipykernel_13504\2091422848.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### Configuration
- 추천 서비스 생성을 위해서 필요한 설정값들을 세팅합니다.

In [2]:
# config
with open('configs/database.yaml') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
from utils.query_service import QueryService
query_service = QueryService(cfg['mysql'])

#### Data Import
- 사용할 데이터를 mysql에서 가져옵니다. 

In [4]:
query_service.query_to_pandas_df('Show tables')

,Tables_in_bakery
0,board_preference_statistic
1,board_statistic
2,food_survey
3,image
4,member
5,member_preference
6,notice
7,preference
8,product
9,product_board


In [5]:
PRODUCT_BOARD_SQL = """
SELECT id
, store_id
, title
, price
, is_soldout
, view 
FROM product_board
"""
product_board = query_service.query_to_pandas_df(PRODUCT_BOARD_SQL)

In [10]:
PRODUCT_SQL = """
SELECT id
, product_board_id
, title as option
, price as option_price
, category
, gluten_free_tag
, high_protein_tag
, sugar_free_tag
, vegan_tag
, ketogenic_tag
FROM product
"""
product = query_service.query_to_pandas_df(PRODUCT_SQL)

In [ ]:
REVIEW_SQL = """
SELECT board_id
, badge_taste
, badge_brix
, badge_texture
, rate
, content
FROM review
"""
review = query_service.query_to_pandas_df(REVIEW_SQL)
review = review.drop_duplicates(subset=['board_id', 'content'], keep='first')

In [ ]:
BOARD_STAT_SQL = """
SELECT board_id
, board_review_grade
, board_view_count
, board_review_count
, board_wish_count 
FROM board_statistic
"""
board_stat = query_service.query_to_pandas_df(BOARD_STAT_SQL)

In [7]:
# board_id가 각 preference_type마다 얼만큼의 preference_score를 갖는지
# 아직 신뢰성있는 데이터로 보이진 않음 (샘플이 적고 score의 분산이 너무 큼)
BOARD_PREFERENCE_STAT_SQL = """
SELECT board_id
, preference_score
, preference_type
, preference_weight
FROM board_preference_statistic
ORDER BY board_id
"""
board_preference_stat = query_service.query_to_pandas_df(BOARD_PREFERENCE_STAT_SQL)

#### Preprocessing

#### Modeling

#### Training

#### Evaluating

#### INFERENCE

#### SAVE

#### 추천 결과 테이블 포맷
- query item은 찜이 눌린 아이템입니다.
- recommendation item은 query 아이템 당 3개를 추출합니다.
- 유사한 recommendation item이 없을 경우 랜덤한 3개의 결과를 추천합니다.
- 포맷에 맞게 데이터프레임을 만듭니다. 

In [24]:
import pandas as pd
from datetime import datetime
data = {
    'query_item': [1, 1, 1, 2, 2, 2],
    'recommendation_item': [2, 3, 5, 6, 7, 8],
    'score': [0.67, 0.54, 0.51, 1.0, 1.0, 1.0],
    'rank': [1, 2, 3, 1, 2, 3],
    'recommendation_theme': ['similarity', 'similarity', 'similarity', 'random', 'random', 'random'],
    'model_version': ['bow-1', 'bow-1', 'bow-1', 'default_model', 'default_model', 'default_model'],
    'created_at': [datetime(2024, 9, 19, 11, 58, 22, 206696)] * 6,
    'modified_at': [datetime(2024, 9, 19, 11, 58, 22, 206696)] * 6
}

recommendation_df = pd.DataFrame(data)


In [25]:
recommendation_df

,query_item,recommendation_item,score,rank,recommendation_theme,model_version,created_at,modified_at
0,1,2,0.67,1,similarity,bow-1,2024-09-19 11:58:22.206696,2024-09-19 11:58:22.206696
1,1,3,0.54,2,similarity,bow-1,2024-09-19 11:58:22.206696,2024-09-19 11:58:22.206696
2,1,5,0.51,3,similarity,bow-1,2024-09-19 11:58:22.206696,2024-09-19 11:58:22.206696
3,2,6,1.00,1,random,default_model,2024-09-19 11:58:22.206696,2024-09-19 11:58:22.206696
4,2,7,1.00,2,random,default_model,2024-09-19 11:58:22.206696,2024-09-19 11:58:22.206696
5,2,8,1.00,3,random,default_model,2024-09-19 11:58:22.206696,2024-09-19 11:58:22.206696
